In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/test-train-imdb/train.csv
/kaggle/input/test-train-imdb/test.csv


In [2]:
from IPython.display import clear_output
# !pip install wandb
clear_output()

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/test-train-imdb/train.csv
/kaggle/input/test-train-imdb/test.csv


In [4]:
!pip install datasets
!pip install peft
!pip install bs4
clear_output()

In [5]:

import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
# import wandb
import os
import re

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
# from trl import SFTTrainer

from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

# import spacy
# import pandas as pd
# import numpy as np
# import nltk
# from nltk.tokenize.toktok import ToktokTokenizer
# import re
# from bs4 import BeautifulSoup
# from contractions import CONTRACTION_MAP
# import unicodedata


# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from nltk import pos_tag
# import string

# tokenizer = ToktokTokenizer()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

2024-02-25 13:33:57.691490: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 13:33:57.691544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 13:33:57.693103: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


device(type='cuda')

In [6]:
from bs4 import BeautifulSoup

train_dataset = pd.read_csv('/kaggle/input/test-train-imdb/train.csv')
test_dataset = pd.read_csv('/kaggle/input/test-train-imdb/test.csv')

train_dataset['label'] = [1 if x=="positive" else 0 for x in train_dataset['sentiment'] ]
test_dataset['label'] = [1 if x=="positive" else 0 for x in test_dataset['sentiment'] ]

train_dataset = train_dataset.drop('sentiment', axis=1)
test_dataset = test_dataset.drop('sentiment', axis=1)

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text



test_dataset['review'] = test_dataset['review'].apply(strip_html_tags)
train_dataset['review'] = train_dataset['review'].apply(strip_html_tags)



train_texts, train_labels = train_dataset['review'], train_dataset['label']
test_texts, test_labels = test_dataset['review'], test_dataset['label']

train_texts.reset_index(drop=True, inplace=True)
test_texts.reset_index(drop=True, inplace=True)

from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

train_texts.reset_index(drop=True, inplace=True)
val_texts.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
val_labels.reset_index(drop=True, inplace=True)

from transformers import GPT2Tokenizer


/tmp/ipykernel_74/1840652599.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [7]:
from transformers import (
    RobertaForSequenceClassification,
    RobertaTokenizer,
    RobertaModel,
)

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", do_lower_case=True)
tokenizer.pad_token = tokenizer.eos_token
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

import torch

class IMDBdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

        # Ensure all encodings and labels have consistent lengths
        assert all(len(val) == len(self.labels) for val in self.encodings.values()), "Encodings and labels lengths do not match."

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = IMDBdataset(train_encodings, train_labels)
test_dataset = IMDBdataset(test_encodings,test_labels)
val_dataset = IMDBdataset(val_encodings, val_labels)

from transformers import AutoTokenizer, GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained(
        model_name, num_labels=2, output_attentions=False, output_hidden_states=False)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)
    
model.to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using 2 GPUs!


DataParallel(
  (module): GPT2ForSequenceClassification(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (score): Linear(in_features=768, out_features=2, bias=False)
  )
)

In [8]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",  # directory where the results and model checkpoints will be saved
    learning_rate=5e-5,  # learning rate for the optimizer
    per_device_train_batch_size=1,  # batch size per device during training
    num_train_epochs=3,  # number of training epochs
    weight_decay=0.01,  # weight decay for regularization
    fp16=True,
    save_steps=2000  # change save_steps from 500 to 2000
)

trainer = Trainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=val_dataset,  # evaluation dataset
)

trainer.train()


Step,Training Loss
500,2.010300
1000,2.231800
1500,2.178200
2000,1.979000
2500,2.271200
3000,2.045100
3500,1.647300
4000,1.988000
4500,1.813000
5000,2.017100


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


KeyboardInterrupt: 

In [12]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification


tokenizer = GPT2Tokenizer.from_pretrained("gpt2", do_lower_case=True)
model = GPT2ForSequenceClassification.from_pretrained("/kaggle/working/results/checkpoint-6000", return_dict=False)
model.to(device)

# Ensure your model is in evaluation mode
model.eval()

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [14]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [15]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm
model.eval()  # Ensure the model is in evaluation mode

# Store predictions and actual labels
predictions = []
actuals = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # Move tensors to the same device as the model
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)  # Only needed if you're also evaluating performance

        outputs = model(input_ids, attention_mask=attention_mask)

        # Assuming you're doing classification and want the highest probability class
        logits = outputs[0]
        predicted_labels = torch.argmax(logits, dim=1)
        predictions.extend(predicted_labels.cpu().numpy())

100%|██████████| 20000/20000 [28:49<00:00, 11.57it/s]


In [16]:
actual_labels = []
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        labels = batch['labels'].to(device)  # Assuming labels are on the same device
        actual_labels.extend(labels.cpu().numpy())

100%|██████████| 20000/20000 [00:17<00:00, 1163.52it/s]


In [17]:
correct_predictions = sum(p == a for p, a in zip(predictions, actual_labels))
accuracy = correct_predictions / len(predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 89.92%


In [18]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef

# Ensure predictions and actual_labels are numpy arrays or compatible formats
precision = precision_score(actual_labels, predictions)
recall = recall_score(actual_labels, predictions)
f1 = f1_score(actual_labels, predictions)
conf_matrix = confusion_matrix(actual_labels, predictions)
mcc = matthews_corrcoef(actual_labels, predictions)

# ROC-AUC score requires probability scores of the positive class, which might need model.predict_proba() or equivalent
# If your model outputs probabilities, you can use:
# roc_auc = roc_auc_score(actual_labels, prediction_probabilities)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Matthews Correlation Coefficient: {mcc:.2f}")
# print(f"ROC-AUC Score: {roc_auc:.2f}")  # Uncomment if you have probability predictions


Precision: 0.87
Recall: 0.94
F1 Score: 0.90
Confusion Matrix:
[[8503 1432]
 [ 583 9482]]
Matthews Correlation Coefficient: 0.80


RAW MODEL

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", do_lower_case=True)
raw_model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
raw_model = raw_model.to(device=device)
raw_model.eval()


# Store predictions and actual labels
raw_predictions = []
raw_actuals = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # Move tensors to the same device as the model
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)  # Only needed if you're also evaluating performance

        outputs = raw_model(input_ids, attention_mask=attention_mask)

        # Assuming you're doing classification and want the highest probability class
        logits = outputs[0]
        raw_predicted_labels = torch.argmax(logits, dim=1)
        raw_predictions.extend(raw_predicted_labels.cpu().numpy())

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 18%|█▊        | 3685/20000 [05:17<23:31, 11.56it/s]